In [5]:
import pandas as pd
import keras as keras
import keras.layers as layers
from keras.datasets import imdb
import tensorflow as tf
import os
import numpy as np
#from keras.utils import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [6]:
npword = np.load('/kaggle/input/arrnpy/arr.npy')
npword = npword.astype(int)
npword

array([[2482296, 2482473, 2482360, ...,       0,       0,       0],
       [2480501, 2482473, 2482419, ...,       0,       0,       0],
       [2420029, 2482468, 2480983, ...,       0,       0,       0],
       ...,
       [2482473, 2482060, 2480380, ...,       0,       0,       0],
       [2482473, 2482255, 2480421, ...,       0,       0,       0],
       [2481911, 2480457, 2482473, ...,       0,       0,       0]])

In [7]:
npword.shape

(96034, 80)

In [9]:
x_train = npword[:2909]
x_test = npword[2909:3638]

In [10]:
data = pd.read_csv('/kaggle/input/textcnn/96text.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96034 entries, 0 to 96033
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          96034 non-null  int64  
 1   publish_time   96034 non-null  object 
 2   view_counts    96034 non-null  object 
 3   summarization  96034 non-null  object 
 4   相关性            96034 non-null  int64  
 5   涨跌             96034 non-null  float64
 6   x              96034 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 5.1+ MB


In [11]:
y = np.array(data['涨跌'])
y_train = y[:2909]
y_test = y[2909:3638]

In [12]:
y_train = tf.keras.utils.to_categorical(y_train, 2)
y_test = tf.keras.utils.to_categorical(y_test, 2)

In [13]:
#得到等长文本
max_word = 80
x_train.shape

(2909, 80)

In [14]:
#获取训练集的总词汇数
vocab_size = np.max([np.max(x_train[i]) for i in range(x_train.shape[0])]) + 1  # 这里1 代表空格，其索引被认为是0。
vocab_size

2482477

In [15]:
from tensorflow.keras.optimizers import Adam
#构建模型
inputs = keras.Input(shape=(80,), name='img')
x = keras.layers.Embedding(input_dim=vocab_size, output_dim=64, input_length=max_word)(inputs)
x = keras.layers.Reshape((80, 64, 1), input_shape=(80, 64))(x)

x1 = keras.layers.Conv2D(filters=32, kernel_size=(5, 32), padding='valid', activation='relu')(x)
#x1 = layers.BatchNormalization()(x1)
x1 = keras.layers.GlobalMaxPool2D()(x1)

x2 = keras.layers.Conv2D(filters=32, kernel_size=(4, 32), padding='valid', activation='relu')(x)
# x2 = layers.BatchNormalization()(x2)
x2 = keras.layers.GlobalMaxPool2D()(x2)

x3 = keras.layers.Conv2D(filters=32, kernel_size=(3, 32), padding='valid', activation='relu')(x)
# x3 = layers.BatchNormalization()(x3)
x3 = keras.layers.GlobalMaxPool2D()(x3)

x = layers.Concatenate(axis=1)([x1, x2, x3])
#x = layers.Concatenate(axis=1)([x1, x2])

outputs = layers.Dense(2, activation='softmax')(x)

model = keras.Model(inputs, outputs, name='txt_cnn')

model.compile(optimizer=Adam(),
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=[keras.metrics.CategoricalAccuracy()])

model.summary()

Model: "txt_cnn"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ img (InputLayer)    │ (None, 80)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 80, 64)    │ 158,878,5… │ img[0][0]         │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 80, 64, 1) │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 76, 33,    │      5,152 │ reshape[0][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 77, 33,    │      4,128 │ reshape[0][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 78, 33,    │      3,104 │ reshape[0][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 32)        │          0 │ conv2d[0][0]      │
│ (GlobalMaxPooling2… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 32)        │          0 │ conv2d_1[0][0]    │
│ (GlobalMaxPooling2… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 32)        │          0 │ conv2d_2[0][0]    │
│ (GlobalMaxPooling2… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 96)        │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
│                     │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 2)         │        194 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 158,891,106 (606.12 MB)

 Trainable params: 158,891,106 (606.12 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.fit(x_train, y_train,
            validation_data=(x_test, y_test),
            epochs=10,
            batch_size=64,
            verbose=1)

Epoch 1/10


I0000 00:00:1725615416.850642     172 service.cc:145] XLA service 0x79001800acc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725615416.850693     172 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1725615416.850697     172 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


 3/46 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - categorical_accuracy: 0.4974 - loss: 0.6934

I0000 00:00:1725615422.291610     172 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


46/46 ━━━━━━━━━━━━━━━━━━━━ 13s 111ms/step - categorical_accuracy: 0.5339 - loss: 0.6901 - val_categorical_accuracy: 0.5624 - val_loss: 0.6821
Epoch 2/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - categorical_accuracy: 0.5548 - loss: 0.6738 - val_categorical_accuracy: 0.6804 - val_loss: 0.6305
Epoch 3/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - categorical_accuracy: 0.7483 - loss: 0.5748 - val_categorical_accuracy: 0.7133 - val_loss: 0.5264
Epoch 4/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - categorical_accuracy: 0.8558 - loss: 0.3995 - val_categorical_accuracy: 0.7449 - val_loss: 0.5099
Epoch 5/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - categorical_accuracy: 0.8901 - loss: 0.3018 - val_categorical_accuracy: 0.7449 - val_loss: 0.5122
Epoch 6/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - categorical_accuracy: 0.9398 - loss: 0.2007 - val_categorical_accuracy: 0.7503 - val_loss: 0.5417
Epoch 7/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - categorical_accuracy: 0.9740 - loss: 0.1229 - val_

In [17]:
# 对文本进行预测
prediction = model.predict(x_test)
prediction

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step 


array([[0.0014207 , 0.99857926],
       [0.5725339 , 0.4274661 ],
       [0.13341638, 0.8665836 ],
       ...,
       [0.1471303 , 0.8528697 ],
       [0.00194655, 0.99805343],
       [0.99344325, 0.0065568 ]], dtype=float32)

In [18]:
prediction0 = model.predict(x_train)
prediction0

91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[4.8279208e-03, 9.9517208e-01],
       [3.8992185e-03, 9.9610078e-01],
       [9.9208248e-01, 7.9175560e-03],
       ...,
       [9.9974602e-01, 2.5391491e-04],
       [9.7153270e-01, 2.8467325e-02],
       [9.9825007e-01, 1.7498885e-03]], dtype=float32)

In [19]:
y_test[:,0]

array([0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0.,
       0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
       0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
       0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1.,
       0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1.,
       0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1.,
       1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0.,
       1., 0., 1., 1., 1.

In [20]:
prediction[:,0]

array([1.42069662e-03, 5.72533906e-01, 1.33416384e-01, 1.71819851e-01,
       9.97840881e-01, 9.77530777e-01, 9.98930156e-01, 4.93013486e-02,
       9.99642491e-01, 9.92773592e-01, 9.92745042e-01, 1.70236197e-03,
       3.89140025e-02, 6.14326156e-04, 8.05835664e-01, 2.60084063e-01,
       9.75079238e-02, 2.56951421e-01, 9.80640411e-01, 9.99625087e-01,
       9.99737680e-01, 3.49687219e-01, 1.08648717e-01, 1.01573244e-02,
       2.34391823e-01, 9.76235032e-01, 7.90212929e-01, 2.00134562e-03,
       6.31519258e-01, 9.93547216e-03, 9.98781860e-01, 9.77022646e-05,
       5.58367837e-03, 6.69309139e-01, 1.61498319e-03, 1.13916933e-03,
       9.99178588e-01, 5.58156637e-04, 6.44269511e-02, 9.67455268e-01,
       7.75267720e-01, 2.47011706e-03, 8.78102660e-01, 7.60372937e-01,
       7.84636319e-01, 9.96699274e-01, 8.64162482e-03, 9.98367369e-01,
       9.65563059e-01, 8.92931640e-01, 2.39756361e-01, 7.06261396e-01,
       8.43474746e-01, 9.87170875e-01, 8.45837593e-01, 5.05667329e-01,
      

In [21]:
pred = np.concatenate((prediction[:,0], prediction0[:,0]))

In [22]:
for i in range(len(prediction)):
    if prediction[i,0] > 0.5:
        prediction[i,0] = 1
    else:
        prediction[i,0] = 0
prediction[:,0]

array([0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1.,
       0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1.,
       1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1.,
       0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1.,
       1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0.,
       1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
       1., 1., 0., 1., 1.

In [23]:
from sklearn.metrics import classification_report
t = classification_report(y_test[:,0], prediction[:,0])
print(t)

              precision    recall  f1-score   support

         0.0       0.71      0.75      0.73       319
         1.0       0.79      0.76      0.78       410

    accuracy                           0.76       729
   macro avg       0.75      0.75      0.75       729
weighted avg       0.76      0.76      0.76       729



In [24]:
model.save('/kaggle/working/text_cnn_model.h5')